# 교차 검증

훈련 데이터셋으로 모델을 훈련하고, 이런저런 매개변수를 만질때마다 테스트 데이터셋으로 성능을 확인하다보면, 결국 테스트 데이터셋에 맞춰진 모델이 되는 게 아닐까?

--> 훈련 세트에서 검증 세트를 떼오자.
- 훈련 세트에서 모델을 훈련하고,
- 모델 수정 시, 검증 세트를 이용하자
- 여러번 매개변수를 만지면서 최적의 모델을 찾자
- 이 모델에서 다시 한 번 훈련 세트 + 검증 세트로 훈련하자
- 최종적으로 딱 한 번만, 테스트 세트로 성능을 평가하자.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv('https://bit.ly/wine_csv_data')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

여기까진 동일한데, 이제 train_input과 train_target을 다시 train_test_split에 넣어서, 검증 세트를 만들자.

여기서도 test_size=0.2로 두어, 훈련 세트의 20%를 검증 세트로 하자.

In [2]:
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)

print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


이렇게 원래 5197개였던 훈련 세트가 4157개로 줄고, 검증 세트가 1040개가 되었다.

이제, 훈련 세트로 모델을 학습시키고, 검증 세트로 모델을 평가해보자.

In [3]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)

print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


과대적합이 있기 때문에, 적절한 매개변수를 찾아서 모델을 개선해야 한다.

그 전에 검증 세트에 관하여 알아야 할 것이 있다.

교차 검증(Cross Validation)
- 검증 세트를 만들면서 훈련 세트가 줄어들었다.
- 보통 많은 데이터를 훈련에 쓸수록 좋은 모델이 되는데,
- 그렇다고 검증 세트를 너무 조금 떼어놓으면, 검증 점수가 불안해진다.
- 교차 검증은 검증 세트를 떼어 내어 평가하는 과정을 여러 번 반복한다.

k-fold 교차 검증
- 데이터를 k등분을 하고, 그 k개의 부분데이터 중 하나를 검증데이터로 이용
- k개의 부분데이터를 모두 번갈아가며 검증데이터로 이용하면서 모델을 훈련
- 보통 5-fold, 10-fold를 사용한다.
    - 이렇게 하면 검증 세트의 크기는 줄어들지만, 각 폴드에서 계산한 검증 점수를 평균하기 때문에 안정된 점수로 생각할 수 있다.

사이킷런에서 제공하는 cross_validate() 함수
- 평가할 모델 객체를 첫 번째 인자로 전달
- 훈련 세트 전체(input, target)를 2, 3번째 인자로 전달
- 매개변수:
    - scoring: 검증에 사용할 평가 지표
        - 분류 모델의 경우는 'accuracy' (정확도)
        - 회귀 모델의 경우는 'r2' (결정계수)
    - cv: 교차 검증 폴드 수 혹은 스플리터 객체(default: 5)
        - 분류 모델의 경우는 'StratifiedKFold' 객체
        - 회귀 모델의 경우는 'KFold' 객체
    - n_jobs: 사용할 CPU 코어 수 (default: 1)
    - return_train_score: 훈련 세트의 점수 반환 여부 (default: False)

In [6]:
from sklearn.model_selection import cross_validate

scores = cross_validate(dt, train_input, train_target) # 직접 검증 데이터를 분리하지 않는다.
print(scores)

{'fit_time': array([0.00940514, 0.00882339, 0.01037383, 0.00736666, 0.00694036]), 'score_time': array([0.00092912, 0.00101805, 0.0007062 , 0.00068259, 0.00062752]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


fit_time: 폴드별 모델 훈련 시간
score_time: 폴드별 검증 시간
test_score: 폴드별 검증 점수

위 예제에서 교차 검증의 최종 점수는 test_score의 점수 5개를 평균하여 얻을 수 있다.

In [7]:
import numpy as np

print(np.mean(scores['test_score']))

0.855300214703487


**주의점**

cross_validate()는 훈련 세트를 적절히 섞어서 폴드를 나누지 않는다.
- 위 예제에서는 train_test_split() 함수가 데이터를 섞어 주었기 때문에, 따로 섞을 필요가 없었지만,
- 그렇지 않은 경우라면, 분할기(spliter)를 지정해주어야 한다.

사이킷런의 분할기는 교차 검증에서 폴드를 어떻게 나눌지 결정한다.
- 앞서 작성하였지만, 아래와 같은 객체를 import하여 cv 매개변수에 전달한다.
    - 분류 모델의 경우는 'StratifiedKFold' 객체
    - 회귀 모델의 경우는 'KFold' 객체

결과적으로 앞서 수행한 교차 검증은 다음 코드와 동일하다.

In [10]:
from sklearn.model_selection import StratifiedKFold # 와인 분류 모델이므로

scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())

print(np.mean(scores['test_score']))

0.855300214703487


만약 훈련 세트를 직접 섞은 후 10-fold 교차 검증을 하려면 다음과 같이 작성한다.

In [11]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)

print(np.mean(scores['test_score']))

0.8574181117533719


KFold 클래스도 동일한 방식으로 사용할 수 있다.

이어서 결정 트리의 매개변수 값을 바꿔가며 가장 성능이 좋은 모델을 찾아보자.

이 때, 테스트 세트를 사용하지 않고, 교차 검증을 통해서 좋은 모델을 고르면 된다.

# 하이퍼파라미터 튜닝

하이퍼파라미터를 튜닝할 때,
- 한 가지 하이퍼파라미터를 최적값으로 취하고, (나머지는 기본값)
- 이어서 다른 하이퍼파라미터의 최적값을 취하는 게

될까?

하이퍼파라미터는 다른 하이퍼파라미터에 영향을 줄 수 있기 때문에, 불가능

그렇다고 수많은 하이퍼파라미터들을 동시에 바꿔가며 최적값을 찾는 건 너무 힘들다.

하지만, 사이킷런에서는 이를 자동으로 수행해주는(AutoML) 그리드 서치(Grid Search) 클래스를 제공한다.

## 그리드 서치

사이킷런의 GridSearchCV 클래스는
- 하이퍼파라미터 탐색과 교차 검증을 한 번에 수행한다. 
    - --> cross_validate() 함수를 따로 호출할 필요가 없다.
- 첫 번째 인자로, 그리드 서치를 수행할 모델 객체를 전달한다.
- 두 번째 인자로, 탐색할 모델의 매개변수와 값을 전달한다.
- 매개변수: scoring, cv, n_jobs, return_train_score 등은 cross_validate() 함수와 동일

기본 매개변수를 사용한 Decision Tree 모델에서, 하이퍼파라미터 중 하나인 min_impurity_decrese의 최적값을 찾아보자.

In [14]:
from sklearn.model_selection import GridSearchCV

params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
# 결정 트리 객체를 생성하자마자 바로 전달

이후로는 일반 모델을 훈련하는 것처럼 fit()을 호출한다.
- 그리드 서치 객체는 결정 트리 모델의 min_impurity_decrease 값을 바꿔가며 총 5번 실행한다.
    - GridSearchCV의 cv의 기본값은 5이므로, 총 25개의 모델을 훈련하는 셈.
    - 따라서, n_jobs를 통해 병렬 실행에 사용할 CPU 코어 수를 지정하는 것이 좋다.

In [15]:
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

이제 이렇게 훈련된 25개의 모델 중, 최적의 하아퍼파라미터를 이용하여

전체 훈련 세트에 대해 다시 한 번 모델을 만들어야 하는데,

다행히도, GridSearchCV는 자기가 알아서 이 일을 대신해준다.
- 그렇게 구해진 모델은 best_estimator_ 속성에 저장되어 있으며,
- 그 모델을 일반적인 결정 트리 모델로 똑같이 사용할 수 있다.

그리고 그리드 서치로 찾은 최적의 매개변수는 best_params_ 속성에 저장되어 있다.

In [17]:
dt = gs.best_estimator_

print(dt.score(train_input, train_target))
print(gs.best_params_)

0.9615162593804117
{'min_impurity_decrease': 0.0001}


모든 하이퍼파라미터가 디폴트값일때, 최적의 min_impurity_decrease는 0.0001으로 선택되었다.

각 매개변수에서 수행한 교차 검증의 평균 점수는 cv_result_속성의 'mean_test_score' 키에 저장되어 있다.

In [18]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


첫 번째 값이 가장 최적인 듯 하다.

매개변수의 후보가 많다면, 이를 수동으로 찾는 것 보다는 np.argmax()를 통해 인덱스를 찾을 수 있다.

그다음, 이 인덱스를 사용하여 params 키에 저장된 매개변수를 출력할 수 있다.

이렇게 얻어진 매개변수가 앞서 출력한 gs.best_params_와 동일한지 확인하자.

In [19]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])

print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


이 과정을 정리하면 다음과 같다.
1. 먼저 탐색할 매개변수를 정한다.
2. 훈련 세트에서 그리드 서치를 수행하여, 최상의 평균 검증 점수가 나오는 매개변수 조합을 찾는다. 이 조합은 그리드 서치 객체에 저장된다.
3. 그리드 서치는 최상의 매개변수에서 전체 훈련 세트를 사용하여 최종 모델을 훈련한다. 이 모델도 그리드 서치 객체에 저장된다.

다음으로 좀 더 복잡한 매개변수 조합을 탐색해보자.
- min_impurity_decrease: 노드를 분할하기 위한 불순도 최소 감소량
- max_depth: 트리의 깊이 제한
- min_samples_split: 노드를 나누기 위한 최소 샘플 수

In [21]:
params = {
    'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),   # 9 후보
    'max_depth': range(5, 20, 1), # range()는 정수만 사용 가능!  # 15 후보
    'min_samples_split': range(2, 100, 10)                       # 10 후보
}
# 총 1350 후보 * 5-fold = 6750 모델!

In [22]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

최상의 매개변수 조합과 최상의 교차 검증 점수를 확인해 보자.

In [23]:
print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}
0.8683865773302731


정말 쉽게 최상의 매개변수를 찾을 수 있어서 좋다!

그런데도 한가지 아쉬운 점은,
- 매개변수의 탐색 간격을 0.0001이라느니 1이라느니 설정한 것에 특별한 근거가 없다는 것.

## 랜덤 서치

매개변수의 값이 수치일 때 값의 범위나 간격을 미리 정하기 어렵고, 

또 너무 많은 매개변수 조건 때문에 시간이 오래 걸리는 경우 --> 랜덤 서치를 사용한다.

랜덤 서치에는 
- 매개변수 값의 목록을 전달하는 것이 아닌,
- 매개변수를 샘플링할 수 있는 확률 분포 객체를 전달한다.

먼저 scipy에서 확률 분포 클래스를 가져오자.

In [27]:
from scipy.stats import uniform, randint
# uniform(실수), randint(정수)는 주어진 범위에서 고르게 값을 뽑는다.
# == '균등 분포에서 샘플링한다.'

rgen = randint(0, 10) # 0~9의 범위로 설정된 randint 객체
rgen.rvs(10) # 그 객체에서 임의로 10개의 수를 뽑아낸다.(샘플링)

array([6, 7, 7, 6, 1, 4, 0, 7, 1, 7])

샘플링이 고르게 이루어지는지 확인하기 위해 이번에는 1000개를 샘플링하여 각 숫자의 개수를 세어보자

In [28]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([105,  99, 101, 103,  97, 104,  99,  98,  87, 107]))

각 숫자가 어느정도 고르게 추출된 것 같다.

uniform 클래스도 동일하다.

In [29]:
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.28402815, 0.42140791, 0.40606469, 0.44220383, 0.80809037,
       0.27382444, 0.45771534, 0.70998632, 0.15399698, 0.43003869])

랜덤 서치 객체에 randint나 uniform 객체를 넘겨주고, 총 몇번을 샘플링해서 최적의 매개변수를 찾으라고 명령할 수 있다.

샘플링 횟수는 시스템 자원이 허락하는 범위 내에서 최대한 크게 하는 것이 좋다.

이제 params 딕셔너리를 다시 만들텐데, 이번에는 아래 매개변수도 추가한다.
- min_samples_leaf: 리프 노드가 되기 위한 최소 샘플의 개수
    - 어떤 노드가 분할하여 만들어질 자식 노드의 샘플 수가, 이 값보다 작으면 분할하지 않는다.

In [31]:
params = {
    'min_impurity_decrease': uniform(0.0001, 0.001), 
    'max_depth': randint(20, 50), 
    'min_samples_split': randint(2, 25),
    'min_samples_leaf': randint(1, 25)
}

샘플링 횟수는 RandomizedSearchCV의 n_iter 매개변수에 지정한다.

In [32]:
from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7ff83a88ed50>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7ff83a77fe50>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7ff83b24e850>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7ff83b24e290>},
                   random_state=42)

최상의 매개변수 조합과 최상의 교차 검증 점수를 확인해 보자.

In [33]:
print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}
0.8695428296438884


이제 gs.best_esimator_에 저장된 모델을 최종 모델로 결정하고, 테스트 데이터셋으로 평가해보자.

In [34]:
dt = gs.best_estimator_

print(dt.score(test_input, test_target))

0.86


아주 만족스럽지는 않지만, 충분히 다양한 매개변수를 테스트해서 얻은 결과임이 자랑스럽다!

# 확인 문제 3

마지막 예제에서 DecisionTreeClassifier 클래스의 splitter='random'을 설정하고 다시 훈련해보자.
- splitter의 기본값은 'best'로 각 노드에서 최선의 분할을 찾는다.
- 'random'은 무작위로 분할한 다음 가장 좋은 것을 고른다.

테스트 세트로 성능을 체크해보고, 왜 이런 옵션이 필요한지 다음 절에서 배워보도록 하자.

In [35]:
gs = RandomizedSearchCV(DecisionTreeClassifier(splitter='random', random_state=42), params, n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42,
                                                    splitter='random'),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7ff83a88ed50>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7ff83a77fe50>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7ff83b24e850>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7ff83b24e290>},
                   random_state=42)

In [36]:
dt = gs.best_estimator_

print(dt.score(test_input, test_target))

0.786923076923077
